In [1]:
print("OK")

OK


In [2]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
%pwd

'e:\\Gen AI\\Source-Code-Analysis-GenAI\\research'

In [4]:
!mkdir test_repo

In [5]:
repo_path = "test_repo/"

repo = Repo.clone_from("https://github.com/entbappy/End-to-end-Medical-Chatbot-Generative-AI", to_path=repo_path)

In [6]:
loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [7]:
documents = loader.load()

In [8]:
documents

[Document(page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_openai import OpenAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\nload_dotenv()\n\nPINECONE_API_KEY=os.environ.get(\'PINECONE_API_KEY\')\nOPENAI_API_KEY=os.environ.get(\'OPENAI_API_KEY\')\n\nos.environ["PINECONE_API_KEY"] = PINECONE_API_KEY\nos.environ["OPENAI_API_KEY"] = OPENAI_API_KEY\n\nembeddings = download_hugging_face_embeddings()\n\n\nindex_name = "medicalbot"\n\n# Embed each chunk and upsert the embeddings into your Pinecone index.\ndocsearch = PineconeVectorStore.from_existing_index(\n    index_name=index_name,\n    embedding=embeddings\n)\

In [9]:
len(documents)

7

In [10]:
documents[0]

Document(page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_openai import OpenAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\nload_dotenv()\n\nPINECONE_API_KEY=os.environ.get(\'PINECONE_API_KEY\')\nOPENAI_API_KEY=os.environ.get(\'OPENAI_API_KEY\')\n\nos.environ["PINECONE_API_KEY"] = PINECONE_API_KEY\nos.environ["OPENAI_API_KEY"] = OPENAI_API_KEY\n\nembeddings = download_hugging_face_embeddings()\n\n\nindex_name = "medicalbot"\n\n# Embed each chunk and upsert the embeddings into your Pinecone index.\ndocsearch = PineconeVectorStore.from_existing_index(\n    index_name=index_name,\n    embedding=embeddings\n)\n

In [16]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 500,
                                                             chunk_overlap = 20)

In [17]:
texts = documents_splitter.split_documents(documents)

In [14]:
texts

[Document(page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_openai import OpenAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\nload_dotenv()', metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='load_dotenv()\n\nPINECONE_API_KEY=os.environ.get(\'PINECONE_API_KEY\')\nOPENAI_API_KEY=os.environ.get(\'OPENAI_API_KEY\')\n\nos.environ["PINECONE_API_KEY"] = PINECONE_API_KEY\nos.environ["OPENAI_API_KEY"] = OPENAI_API_KEY\n\nembeddings = download_hugging_face_embeddings()\n\n\nindex_name = "medicalbot"\n\n# Embed each chunk and upsert the embeddings into your Pin

In [15]:
len(texts)

13

In [12]:
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [13]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [14]:
embeddings=OpenAIEmbeddings(disallowed_special=())

In [18]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./db')

In [19]:
vectordb.persist()

In [20]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()

In [21]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [22]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

In [23]:
question = "what is download_hugging_face_embeddings funtion?"

In [24]:
result = qa(question)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 13, updating n_results = 13


The function `download_hugging_face_embeddings` downloads pre-trained embeddings from the Hugging Face library. In this case, it specifically downloads the model 'sentence-transformers/all-MiniLM-L6-v2' which returns embeddings of 384 dimensions. These embeddings are used for text representation and similarity calculations in the context of the medicalbot application.


In [25]:
question = "what is load_pdf_file funtion?"
result = qa(question)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 13, updating n_results = 13


The purpose of the `load_pdf_file` function is to extract data from PDF files using the `DirectoryLoader` class with a specified file extension of `*.pdf` and the `PyPDFLoader`. The function loads the PDF documents from the directory and returns them for further processing or analysis.
